In [2]:
import pandas as pd
import networkx as nx
# import matplotlib.pyplot as plt
from community import community_louvain
import numpy as np

In [13]:
def load_graph(path, county):

    features_df = pd.read_csv(path + county + '_features.csv', header=None).rename(columns={0: 'GEOID', 1: 'population', 2: 'neighboors'})
    distances_df = pd.read_csv(path + county + '_distances.csv').drop(columns=['Node_ID'])

    distances_df.columns = distances_df.index
    print(features_df)


    pop_dict = features_df['population'].to_dict()
    geoid_dict = features_df['GEOID'].to_dict()
    neighboors_dict = features_df['neighboors'].to_dict()
    print(neighboors_dict)
    print(geoid_dict)

    full_graph = nx.from_pandas_adjacency(distances_df)
    nx.set_node_attributes(full_graph, pop_dict, 'population')
    nx.set_node_attributes(full_graph, geoid_dict, 'geoid')

    print(full_graph.nodes(data=True))

    G = nx.Graph()

    for node, edges in full_graph.adjacency():
        G.add_node(node, **full_graph.nodes[node])
        neighbor_geoids = neighboors_dict.get(node, []).split(',')
        
        # Iterate over the edges
        for neighbor, attributes in edges.items():
            if str(geoid_dict.get(neighbor)) in neighbor_geoids:
                G.add_edge(node, neighbor, **attributes)
    
    return G

In [14]:
G = load_graph('data/AR/counties/', 'AR')

    GEOID  population                                 neighboors
0    5047       17097              05083,05033,05131,05071,05087
1    5033       60133                    05047,05131,05087,05143
2    5067       16755  05111,05037,05145,05063,05147,05075,05031
3    5021       14552                                05055,05121
4    5045      123498  05029,05141,05105,05119,05023,05085,05145
..    ...         ...                                        ...
70   5143      245871                          05033,05087,05007
71   5101        7225        05009,05115,05129,05071,05087,05015
72   5013        4739              05039,05139,05103,05025,05011
73   5037       16833              05123,05111,05035,05147,05067
74   5107       16568                    05077,05095,05041,05001

[75 rows x 3 columns]
{0: '05083,05033,05131,05071,05087', 1: '05047,05131,05087,05143', 2: '05111,05037,05145,05063,05147,05075,05031', 3: '05055,05121', 4: '05029,05141,05105,05119,05023,05085,05145', 5: '05089,05129,

In [6]:
G.edges(data=True)

EdgeDataView([])

In [8]:
from loading import load_graph
G = load_graph('data/RI/')

In [24]:
def evaluate_districts(graph, partition, pop_attr='population'):
    """
    Evaluates the new metric for given districts.
    - graph: The networkx graph
    - partition: A dict mapping node to district
    - pop_attr: The attribute name for population
    Returns: tuple of (population variance, total intra-district distance)
    """
    # Calculate population per district
    district_pops = {}
    for node, district in partition.items():
        pop = graph.nodes[node].get(pop_attr, 0)
        if district in district_pops:
            district_pops[district] += pop
        else:
            district_pops[district] = pop
            
    # Calculate population variance
    pop_variance = np.var(list(district_pops.values()))
    
    # Calculate total intra-district distance per district
    distric_dist = {}
    for u, v, data in graph.edges(data=True):
        if partition[u] == partition[v]:  # Edge within the same district
            district = partition[u]
            if district in distric_dist:
                distric_dist[district] += data.get('weight', 1)
            else:
                distric_dist[district] = data.get('weight', 1)
            
    return pop_variance, distric_dist, district_pops

In [25]:
import random
random.seed(42)
np.random.seed(42)

In [26]:
def calculate_population_variance(graph, partition):
    """
    Calculate the variance of the population across districts in the partition.
    """
    district_populations = {}
    for node, district in partition.items():
        pop = graph.nodes[node].get('population', 0)
        if district not in district_populations:
            district_populations[district] = 0
        district_populations[district] += pop
    
    populations = list(district_populations.values())
    if not populations:  # Prevent division by zero
        return 0
    return np.var(populations)

def calculate_intra_district_distance(graph, partition):
    """
    Calculate the total distance within districts, encouraging compactness.
    """
    total_distance = 0
    for (u, v, data) in graph.edges(data=True):
        if partition[u] == partition[v]:  # Nodes belong to the same district
            total_distance += data.get('weight', 1)  # Assuming 'weight' represents distance
    return total_distance

def custom_metric(graph, partition, balance_factor=0.5):
    """
    Custom metric combining population variance and intra-district distance.
    
    Parameters:
    - graph: The graph object.
    - partition: A dictionary mapping each node to its district.
    - balance_factor: Determines the balance between the two objectives. Range: [0, 1].
                      0 gives all importance to population variance, 1 gives all to compactness.
    
    Returns:
    - A combined metric score where lower is better.
    """
    pop_variance = calculate_population_variance(graph, partition)
    intra_district_distance = calculate_intra_district_distance(graph, partition)
    
    # Normalize metrics to ensure they are on a similar scale
    max_pop_variance = max(pop_variance, 1)  # Prevent division by zero
    max_distance = max(intra_district_distance, 1)  # Prevent division by zero
    
    # Weighted sum of normalized metrics
    metric = ((1 - balance_factor) * (pop_variance / max_pop_variance) + 
              balance_factor * (intra_district_distance / max_distance))
    
    return metric

In [30]:
def aggregate_graph(graph, partition):
    """
    Aggregate the graph based on the current partition, where each community becomes a single node.
    
    Parameters:
    - graph: The original graph.
    - partition: A dictionary mapping each node to its community.
    
    Returns:
    - The aggregated graph where each node represents a community.
    """
    # Create a new graph where each node represents a community
    aggregated_graph = nx.Graph()
    
    # Map each node to its community and aggregate edges within communities
    community_map = {}
    for node, community in partition.items():
        if community not in community_map:
            community_map[community] = {
                'nodes': set(),
                'population': 0,
                'internal_edges': 0,
                'total_edges': 0
            }
        community_map[community]['nodes'].add(node)
        community_map[community]['population'] += graph.nodes[node]['population']
    
    # Add aggregated communities as nodes
    for community, info in community_map.items():
        aggregated_graph.add_node(community, population=info['population'])
    
    # Aggregate edges between communities
    for u, v, data in graph.edges(data=True):
        cu = partition[u]
        cv = partition[v]
        if cu == cv:
            # Internal edge, add to the internal edges count
            community_map[cu]['internal_edges'] += 1
        else:
            # Edge between communities, add or update edge in the aggregated graph
            if aggregated_graph.has_edge(cu, cv):
                aggregated_graph[cu][cv]['weight'] += data['weight']
            else:
                aggregated_graph.add_edge(cu, cv, weight=data['weight'])
                
    return aggregated_graph

In [44]:
def modified_louvain_algorithm(G, custom_metric, iter=0, initial_partition=None):
    if initial_partition is None:
        # Initialize partition with each node in its own community
        partition = {node: node for node in G.nodes()}
    else:
        partition = initial_partition
    
    current_metric = custom_metric(G, partition)

    while iter < 100 and len(set(partition.values())) > 248:
        # Aggregate the graph based on the current partition
        aggregated_graph = aggregate_graph(G, partition)

        # Prepare a new partition map for the aggregated graph
        new_partition = {node: node for node in aggregated_graph.nodes()}

        # Apply the optimization recursively on the aggregated graph
        optimized_partition = modified_louvain_algorithm(aggregated_graph, custom_metric, iter=iter+1, initial_partition=new_partition)

        # Map the optimized partition back to the original graph's nodes
        updated_partition = {}
        for super_node, community in optimized_partition.items():
            for node in partition.keys():
                if partition[node] == super_node:
                    updated_partition[node] = community
        
        # Check if there was an improvement
        new_metric = custom_metric(G, updated_partition)
        if new_metric < current_metric:
            partition = updated_partition
            current_metric = new_metric
            improvement = True
        else:
            improvement = False
    
    return partition

In [45]:
final_partition = modified_louvain_algorithm(G, custom_metric)
print(len(set(final_partition.values())))

In [23]:

# Placeholder for modified Louvain method adaptation

# For now, let's just apply the standard Louvain method for community detection
partition = community_louvain.best_partition(G)
pop_variance, distric_dist, district_pops = evaluate_districts(G, partition)

pop_variance, distric_dist, district_pops

(8380483059.484375,
 {2: 243, 1: 472, 3: 352, 4: 307, 5: 575, 6: 1, 0: 336, 7: 77},
 {2: 218283,
  1: 215527,
  3: 40261,
  4: 95866,
  5: 132267,
  6: 1410,
  0: 290801,
  7: 102964})

In [17]:
list(partition.keys())

int